# Engineering Notebook: Design Exercise 1

### Ideas for the Wire Protocol

- Use TCP/IP as template
    - Do we need IP addresses in the header?
    - How can we verify packets?
- VBS: Should we use some kind of compression for the message contents?
    - VBS: Maybe not, let's keep it simple
- Features:
    - Header version
    - Header checksum
    - Content checksum